<a href="https://www.kaggle.com/code/saugatkafley/binary-machine-failures-randomforest-kfold?scriptVersionId=133583840" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machine-failure-predictions/machine failure.csv
/kaggle/input/playground-series-s3e17/sample_submission.csv
/kaggle/input/playground-series-s3e17/train.csv
/kaggle/input/playground-series-s3e17/test.csv


# Dataset 

In [2]:
sample = pd.read_csv("/kaggle/input/playground-series-s3e17/sample_submission.csv")
train = pd.read_csv("/kaggle/input/playground-series-s3e17/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e17/test.csv")
original = pd.read_csv("/kaggle/input/machine-failure-predictions/machine failure.csv")

print('The dimension of the train synthetic dataset is:', train.shape)
print('The dimension of the test synthetic dataset is:', test.shape)
print('The dimension of the orginal dataset is:', original.shape)
print('The dimension of the submission dataset is:', sample.shape)


The dimension of the train synthetic dataset is: (136429, 14)
The dimension of the test synthetic dataset is: (90954, 13)
The dimension of the orginal dataset is: (10000, 14)
The dimension of the submission dataset is: (90954, 2)


In [3]:
train.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [4]:
sample.head()

,id,Machine failure
0,136429,0.5
1,136430,0.5
2,136431,0.5
3,136432,0.5
4,136433,0.5


In [5]:
train.dtypes == object

id                         False
Product ID                  True
Type                        True
Air temperature [K]        False
Process temperature [K]    False
Rotational speed [rpm]     False
Torque [Nm]                False
Tool wear [min]            False
Machine failure            False
TWF                        False
HDF                        False
PWF                        False
OSF                        False
RNF                        False
dtype: bool

In [6]:
train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
train.Type.unique()

array(['L', 'M', 'H'], dtype=object)

In [8]:
train["Type"] = train.Type.map( {"L":0 , "M":1 , "H":2})
train.Type

0         0
1         1
2         0
3         0
4         1
         ..
136424    1
136425    2
136426    0
136427    0
136428    0
Name: Type, Length: 136429, dtype: int64

In [9]:
test["Type"] = test.Type.map( {"L":0 , "M":1 , "H":2})
test.Type

0        0
1        0
2        0
3        1
4        1
        ..
90949    0
90950    0
90951    0
90952    0
90953    0
Name: Type, Length: 90954, dtype: int64

In [10]:
train["Machine failure"].unique()

array([0, 1])

In [11]:
train.isna().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [12]:
test.shape

(90954, 13)

## Preprocessing

In [13]:
from sklearn.preprocessing import StandardScaler


In [14]:
y_train = train[["Machine failure"]]
y_train.head()

,Machine failure
0,0
1,0
2,0
3,0
4,0


In [15]:
cols = train.columns.to_list()
cols.append(cols.pop(8))

In [16]:
train = train[cols]
test = test[cols[:-1]]

In [17]:
test.shape

(90954, 13)

In [18]:
X_train = train.iloc[:,2:-1]
y_train = train.iloc[:, -1]

In [19]:
X_train.columns

Index(['Type', 'Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'TWF',
       'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [20]:
sc = StandardScaler()
X_train.iloc[:,:] = sc.fit_transform(X_train.iloc[:,:])
X_train.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,-0.608282,0.395880,-0.246230,0.545416,-0.499711,0.556417,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
1,1.051530,1.469856,1.558605,1.720308,-1.323028,1.494433,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
2,-0.608282,-0.302204,-1.040358,2.051873,-1.628831,-1.241447,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
3,-0.608282,0.610675,0.692284,0.026445,0.464745,1.447532,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
4,1.051530,-1.000288,-0.679391,0.869773,-0.582043,-1.100744,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568


In [21]:
X_train.shape

(136429, 11)

In [22]:
y_train.head

<bound method NDFrame.head of 0         0
1         0
2         0
3         0
4         0
         ..
136424    0
136425    0
136426    0
136427    0
136428    0
Name: Machine failure, Length: 136429, dtype: int64>

In [23]:
test.columns 

Index(['id', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'],
      dtype='object')

In [24]:
X_test = test.iloc[:, 2:]
X_test.shape

(90954, 11)

In [25]:
X_test.dtypes

Type                         int64
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object

In [26]:
X_test.iloc[:,:] = sc.transform(X_test.iloc[:,:])
X_test.head()

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,-0.608282,1.308759,1.125444,-0.153753,-0.276240,-0.694271,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
1,-0.608282,0.986567,0.764477,1.388744,-1.358313,-1.366516,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
2,-0.608282,0.771772,0.331317,0.033653,-0.311525,-0.131461,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
3,1.051530,0.127386,-0.246230,-0.297912,0.852880,-1.554119,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568
4,1.051530,1.899446,1.702992,-0.038426,0.111895,0.149943,-0.03945,-0.072021,-0.049016,-0.063038,-0.047568


## Decision Tree Classifier

In [27]:
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss


In [28]:
X_train.shape , y_train.shape

((136429, 11), (136429,))

In [29]:
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

X = X_train.copy()
Y = y_train.copy()
test_baseline = X_test.copy()

rf_cv_scores, rf_preds = list(), list()

skf = StratifiedKFold(n_splits = 10,  shuffle = True , random_state = 42)

for i , (train_ix , test_ix) in enumerate(skf.split(X, Y)):
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
    print('---------------------------------------------------------------')
    rf_model = RandomForestClassifier().fit(X_train , Y_train)
#     gb_model = GradientBoostingClassifier(
#         n_estimators = 500 , 
#         max_depth = 7 , 
#         learning_rate = 0.01, 
#         min_samples_split = 10 , 
#         min_samples_leaf = 20,
#     ).fit(X_train, Y_train)
    rf_pred_1 = rf_model.predict_proba(X_test)[:,1]
    rf_pred_2  =rf_model.predict_proba(test_baseline)[:,1]

    rf_score_fold = roc_auc_score(Y_test , rf_pred_1)
    rf_cv_scores.append(rf_score_fold)
    rf_preds.append(rf_pred_2)
    print('Fold', i, '==> GradientBoositng oof ROC-AUC score is ==>', rf_score_fold)
# clf = RandomForestClassifier()
# clf.fit(X_train , y_train)

---------------------------------------------------------------
Fold 0 ==> GradientBoositng oof ROC-AUC score is ==> 0.9368871802759129
---------------------------------------------------------------
Fold 1 ==> GradientBoositng oof ROC-AUC score is ==> 0.9520424174408213
---------------------------------------------------------------
Fold 2 ==> GradientBoositng oof ROC-AUC score is ==> 0.9565396152433998
---------------------------------------------------------------
Fold 3 ==> GradientBoositng oof ROC-AUC score is ==> 0.9453358480370764
---------------------------------------------------------------
Fold 4 ==> GradientBoositng oof ROC-AUC score is ==> 0.9630797500536886
---------------------------------------------------------------
Fold 5 ==> GradientBoositng oof ROC-AUC score is ==> 0.9482014326191021
---------------------------------------------------------------
Fold 6 ==> GradientBoositng oof ROC-AUC score is ==> 0.9212220559608177
------------------------------------------------

In [30]:
rf_preds

[array([0.  , 0.  , 0.  , ..., 0.01, 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.  , 0.01, 0.  , ..., 0.  , 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.])]

In [31]:
rf_cv_score = np.mean(rf_cv_scores)
rf_cv_score

0.9462728348548447

In [32]:
rf_preds_test = pd.DataFrame(rf_preds).apply(np.mean, axis = 0)

In [33]:
submission = pd.read_csv("/kaggle/input/playground-series-s3e17/sample_submission.csv")
submission['Machine failure'] = rf_preds_test
submission.to_csv("/kaggle/working/submission.csv", index = False)